#  CEvNS rate

In this notebook I will work out how to calculate the CEvNS rate (initially in xenon, following Jay's code), before writing it into a single script

I will start by just taking one of the fluxes calculated from the spectrum generator, but eventually it might be more useful to just define a spectrum function based on that code

The rate calculation takes the following form:
* Integrate over Enu from Enu_min to 8 MeV, using:
* - Enu_min(E_R, isotope properties)
* Integrand multiplies the:
* - flux (fn of Enu)
* - isotope abundance (of Xe)
* - CEvNS rate
* Overall normalisation factor uses the:
* - flux normalisation (fn of distance from reactor)
* - mass of xenon

I will also follow Jay's practice in being consistent with units by making everything in terms of GeV

## Constants

In [6]:
GeV = 1
MeV = 1e-3*GeV
keV = 1e-6*GeV
eV = 1e-9*GeV
hbarc = 0.19732698 # GeV fm
speed_of_light = 2.997925e8 # m/s
Gf = 1.166378e-5/GeV**2
electron_charge = 1.60217663e-19 # coulombs
amu = 0.931494

METER = 1e15/hbarc
CENTIMETER = 1e-2*METER
SECOND = speed_of_light*METER
YEAR = 60*60*24*365.25*SECOND
JOULE = eV/electron_charge
GW = 1e9*JOULE/SECOND
KILOGRAM = speed_of_light**2*JOULE
KILOGRAMDAY = KILOGRAM*(60*60*24*SECOND)
CM2S = CENTIMETER**2*SECOND

In [8]:
xe124 = {"Z": 54, "A": 124, "abundance": 0.00095, "mass": 123.906*amu}
xe126 = {"Z": 54, "A": 126, "abundance": 0.00089, "mass": 125.904*amu}
xe128 = {"Z": 54, "A": 128, "abundance": 0.0191, "mass": 127.904*amu}
xe129 = {"Z": 54, "A": 129, "abundance": 0.26401, "mass": 128.905*amu}
xe130 = {"Z": 54, "A": 130, "abundance": 0.04071, "mass": 129.904*amu}
xe131 = {"Z": 54, "A": 131, "abundance": 0.21232, "mass": 130.905*amu}
xe132 = {"Z": 54, "A": 132, "abundance": 0.26909, "mass": 131.904*amu}
xe134 = {"Z": 54, "A": 134, "abundance": 0.10436, "mass": 133.905*amu}
xe136 = {"Z": 54, "A": 136, "abundance": 0.08857, "mass": 135.907*amu}
isotopesXe = [xe124, xe126, xe128, xe129, xe130, xe131, xe132, xe134, xe136]

mXe = 131.293*amu

## CEvNS cross section

## Fluxes

Eventually the goal will be to write code here that generates spectra for given reactor properties, but for now I will just import fluxes calculated using the spectrum generator code

## CEvNS rate